In [49]:
import sys
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from scipy.stats import pearsonr
import itertools
np.warnings.filterwarnings('ignore')

<h2>1.2.1 Load the data and get an overview of the data</h2>

In [50]:
df = pd.read_csv('ISLR/data/Boston.csv',index_col=0)

In [51]:
r, c = df.shape
print(c)
for a, b in enumerate(df, 1):
    list = "".join('{}. \'{}\''.format(a, b))
    print (list, end=" ")

14
1. 'crim' 2. 'zn' 3. 'indus' 4. 'chas' 5. 'nox' 6. 'rm' 7. 'age' 8. 'dis' 9. 'rad' 10. 'tax' 11. 'ptratio' 12. 'black' 13. 'lstat' 14. 'medv' 

<h1>Assignment 3</h1>

In [53]:
result1 = sm.OLS.from_formula('medv ~ lstat + rm + nox + dis + ptratio', df).fit()
print(result1.summary())

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.708
Model:                            OLS   Adj. R-squared:                  0.705
Method:                 Least Squares   F-statistic:                     242.6
Date:                Wed, 26 Feb 2020   Prob (F-statistic):          3.67e-131
Time:                        19:07:31   Log-Likelihood:                -1528.7
No. Observations:                 506   AIC:                             3069.
Df Residuals:                     500   BIC:                             3095.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     37.4992      4.613      8.129      0.0

In [54]:
result2 = sm.OLS.from_formula('medv ~ lstat * rm + nox + dis + ptratio', df).fit()
print(result2.summary())

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.778
Model:                            OLS   Adj. R-squared:                  0.775
Method:                 Least Squares   F-statistic:                     290.8
Date:                Wed, 26 Feb 2020   Prob (F-statistic):          2.48e-159
Time:                        19:07:33   Log-Likelihood:                -1459.9
No. Observations:                 506   AIC:                             2934.
Df Residuals:                     499   BIC:                             2963.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      3.1518      4.880      0.646      0.5

<h2>Interpretation</h2>

The syntax lstat * rm simultaneously includes lstat, rm, and the interaction term lstat × rm as predictors; it is a shorthand for lstat + rm + lstat : rm, as can be seen in the results table as well.  

**lstat : rm**  
To the left of the ~ is the dependent variable (medv). After the ~, we list the two predictor variables. The * indicates that not only do we want each main effect, but we also want an interaction term between lstat and rm.  
From the results table we see, that individually lstat and rm influence the house price positively, but the interaction lstat:rm negatively.

The R-squared value, which is 77.8% tells us that this model is highly significant.
What can be noticed in the summary is the fact that the features nox, dis, ptratio and lstat:rm have negative values, which means that they are not only significant, but also have a relationship to lowering the price of the house.  
Thus, the conclusion is that the features nox, dis, ptratio and lstat:rm have a relation to lower house prices. Furthermore, the features with negative values, also have a highly significant p-value, which means that it is unlikely that there is no relationship between medv and the given feature.  

On the other hand, the following features have positive values: lstat and rm.  
The conclusion for this is, that for example the houses with more rooms have a relationship with increasing the house value. The p-value for rm for example is also highly significant, which also means that it is unlikely that there is no relationship between the house value and the number of rooms.

In [55]:
result3 = sm.OLS.from_formula('medv ~ lstat * rm + np.square(lstat * rm) + nox + dis + ptratio', df).fit()
print(result3.summary())

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.781
Model:                            OLS   Adj. R-squared:                  0.778
Method:                 Least Squares   F-statistic:                     253.9
Date:                Wed, 26 Feb 2020   Prob (F-statistic):          8.05e-160
Time:                        19:07:58   Log-Likelihood:                -1455.8
No. Observations:                 506   AIC:                             2928.
Df Residuals:                     498   BIC:                             2961.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept                10.55

In [7]:
print(sm.stats.anova_lm(result2, result3))

   df_resid          ssr  df_diff     ss_diff         F    Pr(>F)
0     499.0  9500.381881      0.0         NaN       NaN       NaN
1     498.0  9348.435955      1.0  151.945925  8.094303  0.004623


<h3>General</h3>
Most of the results can be tracked back to the summaries as differences, such as the difference in the degrees of freedom, where in this case the difference is 1, since first model has 6 and the second model has 7 degrees of freedom. Thus the difference is one. df_resid shows the Df Residuals of both models, where the first model has  499.0 and the second one has 498.0. 
For the F-values in this example, and also in all following anova tests, if the value was close to 1, it would mean that there is no relationship between the response and the predictors. If the value is greater than 1 it means that there is a relationship.  

<h2>Interpretation</h2>

The increase of R2 and the low p-value associated with the quadratic term suggests that it leads
to an improved model. The ANOVA test was performed to check whether the quadratic fit is superior to the linear fit. As it is visible, the result shows a degree of freedom difference of 1 (indicating that the more complex model has one additional parameter), and very small p-value. Higher degrees of freedom generally mean larger sample sizes, a higher degree of freedom means more power to reject a false null hypothesis and find a significant result. Furthermore, since the p-value is 0.0046, which is < 0.05, we reject the null hypothesis and conclude that the model which squares lstat * rm (uses np.square(lstat * rm)), thus the result3, is significantly better than the model which uses lstat * rm, result2. So, in this case, we use model result3 as a model preferred over result2. Thus, the conclusion is, that the quadratic fit is superior to the linear fit.  

In [56]:
#the way polynomials actually work, however, this gives a weird coefficients output
result4 = sm.OLS.from_formula('medv ~ lstat * rm +' + '+'.join(['np.power(lstat,' + str(i) + ')' for i in range(1,6)]) + '+ nox + dis + ptratio', df).fit()
#print(result4.summary())

In [57]:
#most similar output to the R example:
poly = lambda x, degree : np.linalg.qr(np.vander(x, degree + 1)[:, ::-1])[0][:, 1:]

result4 = sm.OLS.from_formula('medv ~ lstat * rm + poly(lstat,5) + nox + dis + ptratio', df).fit()
print(result4.summary())

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.792
Model:                            OLS   Adj. R-squared:                  0.787
Method:                 Least Squares   F-statistic:                     188.0
Date:                Wed, 26 Feb 2020   Prob (F-statistic):          1.80e-161
Time:                        20:02:33   Log-Likelihood:                -1443.5
No. Observations:                 506   AIC:                             2909.
Df Residuals:                     495   BIC:                             2956.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            15.7971      5.89

In [18]:
print(sm.stats.anova_lm(result2, result4))

   df_resid          ssr  df_diff     ss_diff         F    Pr(>F)
0     499.0  9500.381881      0.0         NaN       NaN       NaN
1     495.0  8903.772453      4.0  596.609428  8.292038  0.000002


<h2>Interpretation</h2>

The increase of R2 and the low p-value associated with the polynomial term suggests that it leads
to an improved model. The ANOVA test was performed to check whether the polynomial fit is superior to the linear fit. As it is visible, the result shows a degree of freedom difference of 4 (indicating that the more complex model has four additional parameters), and very small p-value. Higher degrees of freedom generally mean larger sample sizes, a higher degree of freedom means more power to reject a false null hypothesis and find a significant result. Furthermore, since the p-value is 0.000002, which is < 0.05, we reject the null hypothesis and conclude that the model polynomial up to 5, thus the result4, is significantly better than the model which uses lstat * rm, result2. So, in this case, we use model result4 as a model preferred over result2. Thus, the conclusion is, that the polynomial fit is superior to the linear fit.  

In [19]:
result5 = sm.OLS.from_formula('medv ~ poly(lstat,5) + rm + np.log(rm) + nox + dis + ptratio',df).fit()
print(result5.summary())
print('\n Anova Test: \n ')
print(sm.stats.anova_lm(result2, result5))

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.804
Model:                            OLS   Adj. R-squared:                  0.800
Method:                 Least Squares   F-statistic:                     202.6
Date:                Wed, 26 Feb 2020   Prob (F-statistic):          7.10e-168
Time:                        18:19:25   Log-Likelihood:                -1428.4
No. Observations:                 506   AIC:                             2879.
Df Residuals:                     495   BIC:                             2925.
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           143.2756     14.09

<h2>Interpretation</h2>

The increase of R2 and the low p-value associated with the other non-linear transofrmations, such as log(X) suggests that it leads to an improved model. The ANOVA test was performed to check whether the log(X) fit is superior to the linear fit. As it is visible, the result shows a degree of freedom difference of 4 (indicating that the more complex model has four additional parameters), and very small p-value. Higher degrees of freedom generally mean larger sample sizes, a higher degree of freedom means more power to reject a false null hypothesis and find a significant result. Furthermore, since the p-value is 0.000000000001190966, which is < 0.05, we reject the null hypothesis and conclude that the model which uses log(X), thus the result5, is significantly better than the model which uses lstat * rm, result2. So, in this case, we use model result5 as a model preferred over result2. Thus, the conclusion is, that the other non-linear transofrmations, such as log(X) fit is superior to the linear fit.  


In [20]:
beat_teacher = sm.OLS.from_formula('medv ~ poly(lstat,5) + rm + np.log(rm) + np.square(lstat * rm) + nox + dis + ptratio',df).fit()
print(beat_teacher.summary())
print('\n Anova Test: \n ')
print(sm.stats.anova_lm(result2, beat_teacher))

                            OLS Regression Results                            
Dep. Variable:                   medv   R-squared:                       0.806
Model:                            OLS   Adj. R-squared:                  0.802
Method:                 Least Squares   F-statistic:                     186.6
Date:                Wed, 26 Feb 2020   Prob (F-statistic):          5.56e-168
Time:                        18:20:08   Log-Likelihood:                -1425.4
No. Observations:                 506   AIC:                             2875.
Df Residuals:                     494   BIC:                             2925.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept               124.74

<h2>Beat the teacher</h2>

The increase of R2 (from teacher's version of 80.4\% to this version 80.6\%) and the low p-value associated with the adding of the np.square(lstat * rm) suggests that it leads to an improved model. The ANOVA test was performed to check whether the new fit is superior to the linear fit. As it is visible, the result shows a degree of freedom difference of 5 (indicating that the more complex model has five additional parameters), and very small p-value. Higher degrees of freedom generally mean larger sample sizes, a higher degree of freedom means more power to reject a false null hypothesis and find a significant result. Furthermore, since the p-value is 0.0000000000003124012, which is < 0.05, we reject the null hypothesis and conclude that the model with the polynomial and np.square(lstat * rm), thus the beat_teacher, is significantly better than the model which uses lstat * rm, result2. So, in this case, we use model beat_teacher as a model preferred over result2. Thus, the conclusion is, that the addition of np.square(lstat * rm) to the result4 fit is even more superior to the linear fit.  

In [21]:
df2 = pd.read_csv('ISLR/data/Carseats.csv',index_col=0)
df2.describe()

,Sales,CompPrice,Income,Advertising,Population,Price,Age,Education
count,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000,400.000000
mean,7.496325,124.975000,68.657500,6.635000,264.840000,115.795000,53.322500,13.900000
std,2.824115,15.334512,27.986037,6.650364,147.376436,23.676664,16.200297,2.620528
min,0.000000,77.000000,21.000000,0.000000,10.000000,24.000000,25.000000,10.000000
25%,5.390000,115.000000,42.750000,0.000000,139.000000,100.000000,39.750000,12.000000
50%,7.490000,125.000000,69.000000,5.000000,272.000000,117.000000,54.500000,14.000000
75%,9.320000,135.000000,91.000000,12.000000,398.500000,131.000000,66.000000,16.000000
max,16.270000,175.000000,120.000000,29.000000,509.000000,191.000000,80.000000,18.000000


In [22]:
print(df2['ShelveLoc'].value_counts())
print('\n')
print(df2['Urban'].value_counts())
print('\n')
print(df2['US'].value_counts())

Medium    219
Bad        96
Good       85
Name: ShelveLoc, dtype: int64


Yes    282
No     118
Name: Urban, dtype: int64


Yes    258
No     142
Name: US, dtype: int64


In [23]:
result6 = sm.OLS.from_formula('Sales ~' + '+'.join(df2.columns.difference(['Sales'])), df2).fit()
print(result6.summary())

                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.873
Model:                            OLS   Adj. R-squared:                  0.870
Method:                 Least Squares   F-statistic:                     243.4
Date:                Wed, 26 Feb 2020   Prob (F-statistic):          1.60e-166
Time:                        18:22:34   Log-Likelihood:                -568.99
No. Observations:                 400   AIC:                             1162.
Df Residuals:                     388   BIC:                             1210.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               5.6606    

<h2>Interpretation</h2>

From, for example, df2\['ShelveLoc'\].value_counts() it is visible that Shelveloc takes on three possible values (Bad, Medium, and Good). Python generates automatically dummy variables for a given feature such as Shelveloc. In this case, the generated dummy for Shelveloc are ShelveLoc\[T.Good\], which takes on a value of 1 if the shelving location is good, else if it is bad it takes on 0. Similarly, if the shelving location is medium, ShelveLoc\[T.Medium\] takes on 1, otherwise 0.  
In the summary output it is visible that the ShelveLoc\[T.Good\] has a positive coefficient, which indicates that if the shelving location is good it is also associated to higher sales. The ShelveLoc\[T.Medium\] has also a positive coefficient, which also indicates that a medium shelving location leads to higher sales than a bad shelving location. However, the medium shelving location means lower sales than a good shelving location as well in this case.  
Other duumy variables are US\[T.Yes\] and Urban\[T.Yes\].  
Urban\[T.Yes\] has a positive coefficient, however a p-value of 0.277, which is > 0.05. This suggests that there isn’t a relationship between the location of the store and the number of sales based on the high p-value of the t-statistic. The coefficient states a positive relationship between Urban\[T.Yes\] and Sales: if the store is in an Urban area, the sales will increase by approximately 123 units.   
Similarly, US\[T.Yes\] has a negative coefficient, however a p-value of 0.220, which is also > 0.05. This suggests that there isn’t a relationship between whether the store is in the US or not and the number of sales based on the high p-value of the t-statistic. The coefficient states a negative relationship between USYes and Sales: if the store is in the US, the sales will decrease by approximately 184 units.  

**Other data**  
The R-squared value, which is 87.3% tells us that this model is highly significant.  
What can be noticed in the summary is that the following features have not only negative coefficients, but some of them also high p -values.  More specifically, features such as:  
US: Whether the store is in the US (Yes/No) (p-val: 0.220)  
Education: Education level at location (p-val: 0.285)  
Price: Price for car seats at each site (p-val: 0.000)  
Age: age level of the population (p-val: 0.000)  

Thus, the conclusion is that features such as US and Education, which a p-value > 0.05 suggests that there isn’t a relationship between whether the store is in the US or not, or the education level at a location and the number of sales. However, all four features have a relation to lower the number of sales. The features Price and Age with negative values also have highly significant p-values, which means that it is unlikely that there is no relationship between the number of sales and the given feature.

On the other hand, the following features have positive values:
ShelveLoc\[T.Good\]  (p-val: 0.000)
ShelveLoc\[T.Medium\] (p-val: 0.000)
Urban\[T.Yes\] (p-val: 0.277)
Advertising (p-val: 0.000)
CompPrice (p-val: 0.000)
Income (p-val: 0.000)
Population (p-val: 0.575)

By looking at the individual p-values, it is visible that there isn’t a relationship between the location of the store (Urban\[T.Yes\]) or regional pop in thousands (Population) and the number of sales.  
Furthermore, good or medium shelve location, local ad budget at each location in 1000s of dollars (Advertising), price charged by competitor at each location (CompPrice), and community income level in 1000s of dollars (Income) all increase the number of sales. 

In [35]:
result7 = sm.OLS.from_formula('Sales ~' + '+'.join(df2.columns.difference(['Sales'])) + ' - Population - Education - Age - Urban - US + Income:Advertising + Price:Age', df2).fit()
print(result7.summary())

                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.870
Model:                            OLS   Adj. R-squared:                  0.868
Method:                 Least Squares   F-statistic:                     328.2
Date:                Wed, 26 Feb 2020   Prob (F-statistic):          2.90e-168
Time:                        18:37:44   Log-Likelihood:                -573.74
No. Observations:                 400   AIC:                             1165.
Df Residuals:                     391   BIC:                             1201.
Df Model:                           8                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               3.2991    

<h2>Beat the teacher</h2>

In [48]:
beat_teacher2 = sm.OLS.from_formula('Sales ~' + '+'.join(df2.columns.difference(['Sales'])) + ' - Population - Education - Age - Urban - US + Income*Advertising + Price*Age', df2).fit()
print(beat_teacher2.summary())

                            OLS Regression Results                            
Dep. Variable:                  Sales   R-squared:                       0.875
Model:                            OLS   Adj. R-squared:                  0.872
Method:                 Least Squares   F-statistic:                     302.7
Date:                Wed, 26 Feb 2020   Prob (F-statistic):          6.46e-170
Time:                        18:54:19   Log-Likelihood:                -566.81
No. Observations:                 400   AIC:                             1154.
Df Residuals:                     390   BIC:                             1194.
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               6.4243    

<h3>Interpretation</h3>

The increase of R2 (from teacher's version of 87.0\% to this version 87.5\%) suggests that it leads to an improved model. Furthermore it is visible, that the degree of freedom increased by one (from 8 to 9 - indicating that the more complex model has one additional parameters), and very small p-value. Higher degrees of freedom generally mean larger sample sizes, a higher degree of freedom means more power to reject a false null hypothesis and find a significant result. Thus, the conclusion is, that the change to 'Income * Advertising + Price * Age' to the beat_teacher2 fit is superior to the result7 fit.  